In [11]:
import pm4py
import sys
import random

import pandas as pd
sys.path.append("../../TaskExecutionTimeMining")

from event_log_transformer import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [12]:
file_path = '../../../data/artificial_log.xes'
event_log = pm4py.read_xes(file_path)
event_log

parsing log, completed traces :: 100%|██████████| 1802/1802 [00:00<00:00, 7021.31it/s]


,concept:name,lifecycle:transition,time:timestamp,org:resource,case:concept:name
0,DIAGNOSIS,SCHEDULE,2020-01-01 03:57:40.044121+00:00,None,0
1,DIAGNOSIS,START,2020-01-01 03:57:40.044121+00:00,Jane,0
2,DIAGNOSIS,COMPLETE,2020-01-01 04:34:23.549454+00:00,Jane,0
3,REPAIR,SCHEDULE,2020-01-01 04:34:23.549454+00:00,None,0
4,REPAIR,START,2020-01-01 04:34:23.549454+00:00,Joe,0
...,...,...,...,...,...
16209,DIAGNOSIS,SCHEDULE,2024-12-29 20:55:45.405475+00:00,None,1801
16210,DIAGNOSIS,START,2024-12-29 20:55:45.405475+00:00,Joe,1801
16211,DIAGNOSIS,COMPLETE,2024-12-29 22:15:50.349701+00:00,Joe,1801
16212,REPAIR,SCHEDULE,2024-12-29 22:15:50.349701+00:00,None,1801


In [13]:
event_log['lifecycle:transition'].unique()


array(['SCHEDULE', 'START', 'COMPLETE'], dtype=object)

In [15]:
start_end_event_log2 = TransformEventLog.start_end_event_log(event_log)
start_end_event_log2 = TransformEventLog.seconds_in_day(start_end_event_log2, 'time:timestamp_start')
start_end_event_log2 = TransformEventLog.day_of_week(start_end_event_log2, 'time:timestamp_start')

transformer = TransformEventLog()
start_end_event_log2 = transformer.add_cumulative_dummies(
    start_end_event_log2, 
    case_col='case:concept:name',
    cols_to_encode=['org:resource', 'concept:name']
)

start_end_event_log2

/Users/ferdinandfuchs/Downloads/TaskExecutionTimeMining-main/src/notebooks/Artificial_Example/../../TaskExecutionTimeMining/event_log_transformer.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + '_complete'] - start_end_event_log[timestamp_name + '_start']
/Users/ferdinandfuchs/Downloads/TaskExecutionTimeMining-main/src/notebooks/Artificial_Example/../../TaskExecutionTimeMining/event_log_transformer.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,lifecycle:transition_complete,time:timestamp_complete,duration,duration_seconds,seconds_in_day,day_of_week,1,Clark,Jane,Joe,Karsten,DIAGNOSIS,QUALITY_CONTROL,REPAIR
2,DIAGNOSIS,START,2020-01-01 03:57:40.044121+00:00,Jane,0,COMPLETE,2020-01-01 04:34:23.549454+00:00,0 days 00:36:43.505333,2203,14260,2,0,0,1,0,0,1,0,0
7,REPAIR,START,2020-01-01 04:34:23.549454+00:00,Joe,0,COMPLETE,2020-01-01 14:30:27.423999+00:00,0 days 09:56:03.874545,35763,16463,2,0,0,1,1,0,1,0,1
12,QUALITY_CONTROL,START,2020-01-01 14:30:27.423999+00:00,Joe,0,COMPLETE,2020-01-01 22:13:45.345445+00:00,0 days 07:43:17.921446,27797,52227,2,0,0,1,2,0,1,1,1
17,DIAGNOSIS,START,2020-01-01 08:16:35.844753+00:00,Jane,1,COMPLETE,2020-01-01 08:47:14.772217+00:00,0 days 00:30:38.927464,1838,29795,2,0,0,1,0,0,1,0,0
22,REPAIR,START,2020-01-01 08:47:14.772217+00:00,Karsten,1,COMPLETE,2020-01-01 13:27:22.316694+00:00,0 days 04:40:07.544477,16807,31634,2,0,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26997,QUALITY_CONTROL,START,2024-12-26 19:03:41.622915+00:00,Joe,1799,COMPLETE,2024-12-26 21:39:17.312456+00:00,0 days 02:35:35.689541,9335,68621,3,1,0,0,1,1,1,1,1
27002,DIAGNOSIS,START,2024-12-26 18:05:59.540931+00:00,Clark,1800,COMPLETE,2024-12-26 19:13:26.892809+00:00,0 days 01:07:27.351878,4047,65159,3,0,1,0,0,0,1,0,0
27007,REPAIR,START,2024-12-26 19:13:26.892809+00:00,Clark,1800,COMPLETE,2024-12-27 08:23:46.946760+00:00,0 days 13:10:20.053951,47420,69206,3,0,2,0,0,0,1,0,1
27012,QUALITY_CONTROL,START,2024-12-27 08:23:46.946760+00:00,Jane,1800,COMPLETE,2024-12-27 11:10:44.009123+00:00,0 days 02:46:57.062363,10017,30226,4,0,2,1,0,0,1,1,1


In [18]:
random.seed(42)

train, test = pm4py.ml.split_train_test(start_end_event_log2, case_id_key='case:concept:name')

In [19]:
print(train.shape, test.shape)
print(train['case:concept:name'].unique().shape, test['case:concept:name'].unique().shape)

(4330, 19) (1074, 19)
(1444,) (358,)


In [20]:
train.to_csv('../../../data/transformed_event_logs/AR_train.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')
train.to_pickle('../../../data/transformed_event_logs/AR_train.pickle')

In [21]:
test.to_csv('../../../data/transformed_event_logs/AR_test.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')
train.to_pickle('../../../data/transformed_event_logs/AR_test.pickle')